In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('Lin_Reg').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
#Load data
training = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [0]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [0]:
#Creating model instance
#featuresCol -> tells what the features column is called, same for labelCol, predictionCol
lr = LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')

In [0]:
#Fit and train the model
lrModel = lr.fit(training)

In [0]:
lrModel.coefficients

Out[8]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [0]:
training_summary = lrModel.summary

In [0]:
training_summary.rootMeanSquaredError

Out[11]: 10.16309157133015

In [0]:
#train-test split
all_data = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [0]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                353|
|   mean|0.15109283331045478|
| stddev| 10.418093145986854|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [0]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                148|
|   mean| 0.5092265524398679|
| stddev| 10.105257124658939|
|    min|-26.805483428483072|
|    max| 27.111027963108548|
+-------+-------------------+



In [0]:
new_model = lr.fit(train_data)

In [0]:
test_preds = new_model.evaluate(test_data)

In [0]:
test_preds.rootMeanSquaredError

Out[22]: 10.411125713012693

In [0]:
#Deploying the trained model
unlabeled_data = test_data.select('features')

In [0]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [0]:
predictions = new_model.transform(unlabeled_data)

In [0]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| 1.5337735115881046|
|(10,[0,1,2,3,4,5,...|-1.1650068076755775|
|(10,[0,1,2,3,4,5,...|  5.217951344340831|
|(10,[0,1,2,3,4,5,...|-2.4180546642161396|
|(10,[0,1,2,3,4,5,...| 1.0307362944972807|
|(10,[0,1,2,3,4,5,...|  2.986002216616562|
|(10,[0,1,2,3,4,5,...| 0.4422647817048623|
|(10,[0,1,2,3,4,5,...| 2.7727848368816193|
|(10,[0,1,2,3,4,5,...|-0.3346847778794765|
|(10,[0,1,2,3,4,5,...| 2.0950386943961288|
|(10,[0,1,2,3,4,5,...|-0.5688649378604554|
|(10,[0,1,2,3,4,5,...|  5.715258880001533|
|(10,[0,1,2,3,4,5,...| -2.778083696221436|
|(10,[0,1,2,3,4,5,...| 3.5875498614992525|
|(10,[0,1,2,3,4,5,...|-2.2484133249987566|
|(10,[0,1,2,3,4,5,...|  4.265662737350432|
|(10,[0,1,2,3,4,5,...|-1.2911119164440417|
|(10,[0,1,2,3,4,5,...| 3.1560639294786137|
|(10,[0,1,2,3,4,5,...|-3.0895337337071327|
|(10,[0,1,2,3,4,5,...| -2.060866811740546|
+----------